In [4]:
# %env source=/home/xps/educate/code/hust/XQA/source
# %env src=cc
# %env dir=cc
# from utils import QA_metrics
from transformers import AutoTokenizer

/home/xps/anaconda3/envs/deeplearning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [1]:
# these functions are heavily influenced by the HF squad_metrics.py script

def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""
    
    gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example - 
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]
        
    return gold_answers

In [11]:
def single_f1(a_tokens, b_tokens, tokenizer):
        pred = tokenizer.decode(a_tokens)
        true = tokenizer.decode(b_tokens)

        sum = 0
        if len(pred.split()) == 0 or len(true.split()) == 0:
                return int(pred == true)
        
        text = pred if len(pred.split()) < len(true.split()) else true
        for i in range(len(text.split())):

            print(pred.split()[i], true.split()[i])
            if pred.split()[i] in true.split():
               
                sum += 1

           
  
        precision = sum/len(pred.split())
        recall = sum/len(true.split())
        if precision == 0 or recall == 0:
            return 0
        return 2/(1/precision + 1/recall)

In [15]:
a = "what the fuck is this"
b = ""
a_token, b_token = tokenizer(a).input_ids[1:-1], tokenizer(b).input_ids[1:-1]
compute_f1(a, b), single_f1(a_token, b_token, tokenizer)

(1, 1)